---
## Reddit Data

    In this script I'll explore the data received by Joao on 27th nov 23 and do some analysis for query expansion as it was done for Google Patents
    https://www.kaggle.com/code/jyingong/data-cleaning-sentiment-analysis-reddit
    https://github.com/LoLei/redditcleaner
---

In [45]:
pip install redditcleaner

Note: you may need to restart the kernel to use updated packages.


In [46]:
import pandas as pd #data handling
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
import string
from nltk.probability import FreqDist
import numpy as np
import glob

from collections import Counter
from collections import defaultdict
import matplotlib.pyplot as plt # plotting

nltk.download('stopwords')
nltk.download('punkt')

from gensim.models import Word2Vec
import re  #  preprocessing
from time import time  # time operations
from collections import defaultdict  # word frequency
from sklearn.decomposition import PCA #dimension reduction

#import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", 
                    datefmt= '%H:%M:%S', level=logging.INFO)

# Import needed packages for preprocessing
import re
import string                    
nltk.download('wordnet')
from sklearn import preprocessing

#clean reddit text
import redditcleaner

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/trinidadbosch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/trinidadbosch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/trinidadbosch/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
reddit = pd.read_csv("data/query_reddit_nov23.csv")

In [3]:
reddit.columns

Index(['dataset', 'created_utc', 'retrieved_utc', 'num_comments', 'score',
       'upvote_ratio', 'num_crossposts', 'gilded', 'subreddit_subscribers',
       'stickied', 'locked', 'quarantine', 'over_18', 'id', 'author',
       'author_flair_text', 'link_flair_text', 'subreddit', 'domain', 'title',
       'selftext'],
      dtype='object')

In [3]:
#(reddit)

In [48]:
#Select few columns to work
reddit2 = reddit.iloc[:, [0,1,8,13,14,16,17,18,19,20]]

In [6]:
#reddit2

In [37]:
reddit2.isna().sum()

dataset                       0
created_utc                   0
subreddit_subscribers     92895
id                            0
author                        0
link_flair_text          342921
subreddit                   310
domain                     3502
title                         0
selftext                      0
dtype: int64

In [49]:
## columns to string
reddit2.dtypes
reddit2['title'] = reddit2['title'].astype(str)
reddit2['selftext'] = reddit2['selftext'].astype(str)
reddit2['author'] = reddit2['author'].astype(str)

<ipython-input-49-b7adbb2db0b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit2['title'] = reddit2['title'].astype(str)
<ipython-input-49-b7adbb2db0b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit2['selftext'] = reddit2['selftext'].astype(str)
<ipython-input-49-b7adbb2db0b5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [50]:
# Apply (map) the function on all body column entries
reddit2['selftext'] = reddit2['selftext'].map(redditcleaner.clean)
reddit2

<ipython-input-50-093e26b9115b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit2['selftext'] = reddit2['selftext'].map(redditcleaner.clean)


,dataset,created_utc,subreddit_subscribers,id,author,link_flair_text,subreddit,domain,title,selftext
0,2023-03,1679681905,351036.0,120syxa,mieplol,Production question,videoediting,self.VideoEditing,Best motion interpolation AI right now?,I saw lots of deep learning software which did...
1,2023-03,1679681964,215904.0,120szzj,[deleted],Machine Learning,artificial,NaN,A new artificial intelligence search engine wh...,[removed]
2,2023-03,1679682059,98.0,120t1ss,echojobs,NaN,golangjob,echojobs.io,Motional is hiring Principal Motion Planning E...,nan
3,2023-03,1679682205,35.0,120t4iv,hotentrancetrain,AI trading opprtunities,aitradingopportunity,self.AItradingOpportunity,Machine Learning Basics for Trading and Investing,\n\nLet's break down the basics of machine lea...
4,2023-03,1679682314,0.0,120t6fd,vinaykarthik2,NaN,u_vinaykarthik2,self.VinayKarthik2,Intelligent Process Automation Services,are a type of business solution that combines...
...,...,...,...,...,...,...,...,...,...,...
475725,2023-03,1679674517,379476.0,120p2v5,perseus_dg,NaN,gradschool,self.GradSchool,Confused about choosing a robotics program .,Hello \nI got an admit into a masters program ...
475726,2023-03,1679675432,2767308.0,120pijw,[deleted],Fan Creations,starwars,NaN,"I am obsessed with the Cantina aliens, so I ma...",[removed]
475727,2023-03,1679675459,2.0,120pj2f,n0slandbotmod,NaN,n0sland,news.google.com,Artificial Intelligence is booming – but how w...,nan
475728,2023-03,1679675459,2.0,120pj2o,n0slandbotmod,NaN,n0sland,news.google.com,ChatGPT and artificial intelligence tools coul...,nan


In [52]:
reddit2.selftext

0         I saw lots of deep learning software which did...
1                                                 [removed]
2                                                       nan
3         \n\nLet's break down the basics of machine lea...
4          are a type of business solution that combines...
                                ...                        
475725    Hello \nI got an admit into a masters program ...
475726                                            [removed]
475727                                                  nan
475728                                                  nan
475729    As a software engineer, the best skill set to ...
Name: selftext, Length: 475730, dtype: object

In [53]:
reddit.selftext

0         I saw lots of deep learning software which did...
1                                                 [removed]
2                                                       NaN
3          \n\nLet's break down the basics of machine le...
4         [Intelligent Process Automation services](http...
                                ...                        
475725    Hello \nI got an admit into a masters program ...
475726                                            [removed]
475727                                                  NaN
475728                                                  NaN
475729    As a software engineer, the best skill set to ...
Name: selftext, Length: 475730, dtype: object

In [9]:
reddit2.drop_duplicates(subset=['title'], inplace=True)

<ipython-input-9-23fa3aea15a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit2.drop_duplicates(subset=['title'], inplace=True)


In [14]:
len(reddit2)
reddit.selftext

0         I saw lots of deep learning software which did...
1                                                 [removed]
2                                                       NaN
3          \n\nLet's break down the basics of machine le...
4         [Intelligent Process Automation services](http...
                                ...                        
475725    Hello \nI got an admit into a masters program ...
475726                                            [removed]
475727                                                  NaN
475728                                                  NaN
475729    As a software engineer, the best skill set to ...
Name: selftext, Length: 475730, dtype: object

In [11]:
#split sentences by delimited "\n\n"
reddit2["selftext"] = [item.split("\n\n") for item in reddit2.selftext]

<ipython-input-11-eb2d3e7c4d02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit2["selftext"] = [item.split("\n\n") for item in reddit2.selftext]


In [15]:
reddit2.selftext

0         [I saw lots of deep learning software which di...
1                                               [[removed]]
2                                                     [nan]
3         [ \n\nLet's break down the basics of machine l...
4         [[Intelligent Process Automation services](htt...
                                ...                        
475722                                                [nan]
475725    [Hello \nI got an admit into a masters program...
475726                                          [[removed]]
475728                                                [nan]
475729    [As a software engineer, the best skill set to...
Name: selftext, Length: 332153, dtype: object

In [28]:
#explode nested list into individual rows 
reddit2 = reddit2.explode("selftext")

In [17]:
#replace double space with empty string
reddit2["selftext"] = reddit2.selftext.str.replace("&#x200B;", "")

In [16]:
#reddit2.loc[reddit2.selftext.str.contains("https")]

In [91]:
#see table of items with https links and markup links 
# create a table to see these authors
#df_temp_https = reddit2[reddit2.selftext.str.contains("https")]

In [18]:
# define function to remove both links and markup links 
# also remove \' from dont\'t
def remove_https(item):
    #remove https links
    item_1 = re.sub(r"[(+*)]\S*https?:\S*[(+*)]", "", item)
    #remove https links with no brackets
    item_2 = re.sub('http://\S+|https://\S+', " ", item_1)
    #remove link markups []
    #note that this will also remove comment fields with ["Delete"] 
    item_3 = re.sub(r"[\(\[].*?[\)\]]", " ", item_2)
#     #remove \ in don\'t
#     item_4 = re.sub("[\"\']", "'", item_3)
    return item_3

reddit2["selftext"] = [remove_https(x) for x in reddit2.selftext]
## Do the same for titles?

In [19]:
reddit2.selftext

0         I saw lots of deep learning software which did...
1                                                          
2                                                       nan
3          \n\nLet's break down the basics of machine le...
4           are a type of business solution that combine...
                                ...                        
475722                                                  nan
475725    Hello \nI got an admit into a masters program ...
475726                                                     
475728                                                  nan
475729    As a software engineer, the best skill set to ...
Name: selftext, Length: 332153, dtype: object

In [94]:
#check the temporary table to see if links/ markuplinks/ \' 
#all links has been removed
#unecessary comments (highlighted in yellow) can be removed later by filtering out unecessary usernames
df_temp_https["selftext"] = [remove_https(x) for x in df_temp_https.selftext]
# df_temp.style.apply(lambda x: ['background: lightyellow' if x.username == "RemindMeBot" \
                               # or x.author =="sneakpeek_bot" else '' for i in x], axis=1)

<ipython-input-94-c4ad84d3f099>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_https["selftext"] = [remove_https(x) for x in df_temp_https.selftext]


In [47]:
#reddit2

In [35]:
#reddit2[reddit2.author.isna()]

# getting the bots authors for analysis of thos comments
reddit_bots = reddit2[reddit2['author'].str.contains('bots', case=False, na=False)]

# drop bots authors from reddit dataset
reddit_work = reddit2[~reddit2['author'].str.contains('bot', case=False, na=False)]

In [36]:
reddit_bots

,dataset,created_utc,subreddit_subscribers,id,author,link_flair_text,subreddit,domain,title,selftext
931,2023-03,1679911875,2932.0,123idw3,neurobotsil,NaN,zepeto_official,reddit.com,AI bot for Zepeto. Cybersista,"\nWe visited the premises of ""Neurobots"", a co..."
3199,2023-03,1680184194,0.0,126noi3,neurobotsil,NaN,u_neurobotsil,i.redd.it,Controlled AI Bots for the Gaming Industry,Controlled AI bots are a new direction in the ...
6762,2012-04,1334448709,NaN,sa5va,robotsrobotschicken,NaN,machinelearning,self.MachineLearning,Writing a paper on artificial intelligence,So I'm writing a very basic introductory paper...
9345,2013-06,1370623510,NaN,1fvddx,stupidrobots,Intro,random_acts_of_amazon,self.Random_Acts_Of_Amazon,"[INTRO]I've actually already gifted but hello,...",I love gift exchanges. I love giving things a...
10950,2013-12,1388387532,NaN,1u01wu,alsorobots,NaN,askreddit,self.AskReddit,"Reddit, if you were a super-villain, what woul...",I assume it involves genetics or cloning or po...
...,...,...,...,...,...,...,...,...,...,...
449977,2023-01,1674827314,0.0,10ml8sj,chatbots2023,NaN,u_chatbots2023,self.chatbots2023,CHATGPT (CHAT.OPENAI) EXPLAINS HOW COUNTRIES C...,\n\n\n### CHATGPT (CHAT.OPENAI) EXPLAINS H...
450078,2023-01,1674684786,16702.0,10lbdb3,botsondemand,NaN,nocode,self.nocode,Microsoft and OpenAI Partnership: Impact on AI...,"Hey everyone, I'm curious to hear your thought..."
450449,2023-01,1674901346,0.0,10nby1p,botspice,NaN,u_botspice,self.botspice,How Can You Boost Customer Conversion Using Ch...,Every business thrives on customer conversion ...
460114,2023-02,1676760441,27432.0,115ufp0,robotswanttokillme,NaN,lexfridman,theguardian.com,‘I want to destroy whatever I want’: Bing’s AI...,nan


## Text cleaning
---

In [107]:
# ## CHANGE THIS TO A FUNCTION!!!


# #ensure that comment attribute is of correct data type
# reddit2["selftext"] = reddit2.selftext.astype("str")
# reddit2["selftext"] = [item.lower() for item in reddit2.selftext]

# #remove apostrophe at the beginning and end of each word (e.g. 'like, 'this, or', this')
# reddit2["selftext"] = [re.sub(r"(\B'\b)|(\b'\B)", ' ', item) for item in reddit2.selftext]
# reddit2["selftext"] = [re.sub(r'…', ' ', item) for item in reddit2.selftext]
# reddit2["selftext"] = [item.replace('\\',' ') for item in reddit2.selftext]
# reddit2["selftext"] = [item.replace('/',' ') for item in reddit2.selftext]

In [54]:
def remove_links(comment: str) -> str:
    '''
    Function takes a comment and removes http, bitly links from it. Afterwards returning the comment string.
    '''
    
    # Removing http links
    comment = re.sub(r'http\S+', '', comment)   
    
    # Removing bitly links
    comment = re.sub(r'bit.ly/\S+', '', comment)  
    
    # Removing link mentioning 
    comment = comment.strip('[link]')   # remove [links]
    
    # Removing links to pictures
    comment = re.sub(r'pic.twitter\S+','', comment)
    
    # Returning tweet without link mentions
    return comment


def remove_audiovisual(comment: str) -> str:
    '''
    This function takes a tweet and removes videos and audio tags or labels. Afterwards returning the cleaned tweet.
    '''
    
    # Removing 'VIDEO:' from start of a tweet
    comment = re.sub('VIDEO:', '', comment)  
    
    # Removing 'AUDIO:' from start of a tweet
    comment = re.sub('AUDIO:', '', comment)  
    
    # Returning tweet without videos and audio tags or labels
    return comment


def nlp_preprocess(comments: pd.Series) -> list:
    ''' 
    This NLP function cleans tweets so that they resemble sentences used in the daily language, thus for further analysis no tweet specific trained models are necessray to use. 
    '''
    
    # Creating an empty list to fill with preprocessed tweets
    comments_preprocessed = []

    for comment in comments:
        # Removing urls, hashtags, mentions, reserved words (RT, FAV), emojis, and smileys
        #comment = pre.clean(comment)
        
        # Applying predefined functions for tweet cleaning
        comment = remove_links(comment)
        comment = remove_audiovisual(comment)
        
        # Removing number mentionings
        comment = re.sub('([0-9]+)', '', comment)  
        
        # Removing words containing numbers
        comment = re.sub('\w*\d\w*', '', comment)  
        
        # Removing line break 
        comment = re.sub('\n', '', comment)
        
        # Strip punctuation special to tweets, but uncommon in natural language sentences
        punctuation = '"$%&\'*+:;/()<=>[\\]^_`{|}~•@£'
        comment = re.sub('[' + punctuation + ']+', ' ', comment)  
        
        # Removing double/multiple spacing
        comment = re.sub('\s+', ' ', comment)   
        
        # Normalize characters, means concretly --> todaaaaay = today
        comment = re.sub(r'([A-Za-z])\1{2,}', r'\1', comment) 
        
        # Appending preprocessed tweet to the list tweets_preprocessed. If tweet now empty append NaN
        if comment != '' and comment != ' ':
            comments_preprocessed.append(comment)
        else:
            comments_preprocessed.append('NaN') 
    
    # Returning list of processed tweets
    return comments_preprocessed

In [ ]:
# Applying nlp_preprocess() function to the tweets column to preprocess the data for further analysis
#reddit_bots['selftext'] = nlp_preprocess(reddit_bots['selftext'])  
reddit2['selftext'] = reddit2['selftext'].apply(nlp_preprocess)
#reddit_work['selftext'] = nlp_preprocess(reddit_work['selftext']) 

In [42]:
stop_words = set(stopwords.words('english'))
stop_words.add("None")

def preprocessing(text):
    if isinstance(text, str) and text.lower() != "none":
        words = word_tokenize(text.lower())  # Tokenize the text and convert to lowercase
        filtered_words = [word for word in words if word not in stop_words and not any(char.isdigit() for char in word) and word not in string.punctuation]  # Remove stopwords, numbers, and punctuation
        return filtered_words
    return []

In [43]:
reddit_bots['selftext'] = reddit_bots['selftext'].apply(preprocessing)

<ipython-input-43-9690f20d94b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_bots['selftext'] = reddit_bots['selftext'].apply(preprocessing)


In [44]:
reddit_bots

,dataset,created_utc,subreddit_subscribers,id,author,link_flair_text,subreddit,domain,title,selftext
931,2023-03,1679911875,2932.0,123idw3,neurobotsil,NaN,zepeto_official,reddit.com,AI bot for Zepeto. Cybersista,"[\nwe, visited, premises, neurobots, company, ..."
3199,2023-03,1680184194,0.0,126noi3,neurobotsil,NaN,u_neurobotsil,i.redd.it,Controlled AI Bots for the Gaming Industry,"[controlled, ai, bots, new, direction, field, ..."
6762,2012-04,1334448709,NaN,sa5va,robotsrobotschicken,NaN,machinelearning,self.MachineLearning,Writing a paper on artificial intelligence,"[writing, basic, introductory, paper, artifici..."
9345,2013-06,1370623510,NaN,1fvddx,stupidrobots,Intro,random_acts_of_amazon,self.Random_Acts_Of_Amazon,"[INTRO]I've actually already gifted but hello,...","[love, gift, exchanges, love, giving, things, ..."
10950,2013-12,1388387532,NaN,1u01wu,alsorobots,NaN,askreddit,self.AskReddit,"Reddit, if you were a super-villain, what woul...","[assume, involves, genetics, cloning, possibly..."
...,...,...,...,...,...,...,...,...,...,...
449977,2023-01,1674827314,0.0,10ml8sj,chatbots2023,NaN,u_chatbots2023,self.chatbots2023,CHATGPT (CHAT.OPENAI) EXPLAINS HOW COUNTRIES C...,"[\n\n\n, chatgpt, chat.openai, explains, count..."
450078,2023-01,1674684786,16702.0,10lbdb3,botsondemand,NaN,nocode,self.nocode,Microsoft and OpenAI Partnership: Impact on AI...,"[hey, everyone, curious, hear, thoughts, recen..."
450449,2023-01,1674901346,0.0,10nby1p,botspice,NaN,u_botspice,self.botspice,How Can You Boost Customer Conversion Using Ch...,"[every, business, thrives, customer, conversio..."
460114,2023-02,1676760441,27432.0,115ufp0,robotswanttokillme,NaN,lexfridman,theguardian.com,‘I want to destroy whatever I want’: Bing’s AI...,[]
